# **What is the most optimal skill to learn for Data Analysts?**

### **Methodology**
1. Continue from last notebook to find percent of postings with skill
2. Visualize median salary vs percent skill demand
3. (Optional) Determine if certain technologies are more prevalent

### **Original Exploration**
[15_Matplotlib_Scatter_Plots.ipynb](https://github.com/adinramaadin/Python_Project-2_Data-Job-Market/blob/main/Exploring_the_Data/14_Matplotlib_Scatter_Plots.ipynb)

[16_Matplotlib_Advanced_Customization.ipynb](https://github.com/adinramaadin/Python_Project-2_Data-Job-Market/blob/main/Exploring_the_Data/15_Matplotlib_Advanced_Customization.ipynb)